# Ödev İçin Kod

In [91]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

In [92]:
data = pd.read_csv('tenis.csv')
data

,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


In [93]:
#hava durumu ile ilgili düzenleme
hava = data.iloc[:, 0:1].values
le = preprocessing.LabelEncoder()
hava[:, 0] = le.fit_transform(data.iloc[:,0])

ohe = preprocessing.OneHotEncoder()
hava = ohe.fit_transform(hava).toarray()

#windy ile ilgili düzenleme
windy = data.iloc[:, 3:4].values
windy = ohe.fit_transform(windy).toarray()

#oynanıp oynanmadığı ile ilgili düzenleme
play = data.iloc[:, -1:].values
play = ohe.fit_transform(play).toarray()


print("Hava\n", hava)
print("\n\nRüzgar\n", windy)
print("\n\nOynandı mı\n", play)

Hava
 [[0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]]


Rüzgar
 [[1. 0.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]]


Oynandı mı
 [[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [1. 0.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [94]:
#df oluşturalım
df1 = pd.DataFrame(data=hava, index=range(14), columns=['overcast', 'rainy', 'sunny'])

df2 = pd.DataFrame(data=data.iloc[:, 1:3].values, index=range(14), columns=['temperature', 'humidity'])

#rüzgarlıdan sadece false kolonu alınıyor ki dummy variable olmasın
df3 = pd.DataFrame(data=windy[:, 0:1], index=range(14), columns=['No Wind'])

#aynı şekilde oynanıp oynanmadığı konusunda da dummy variable oluşturmamaya çalışıyoruz. Sadece yes'i alıyoruz
df4 = pd.DataFrame(data=play[:, 1:], index=range(14), columns=['Played'])

#en son tüm df'leri birleştiriyoruz
tumdf = pd.concat([df1, df2, df3, df4], axis=1)
tumdf

#diyelim humidity'i tahmin ettireceğiz
hum = tumdf.iloc[:, 4:5]
sol = tumdf.iloc[:, :4]
sag = tumdf.iloc[:, 5:]

gereklidf = pd.concat([sol, sag], axis=1)
gereklidf

,overcast,rainy,sunny,temperature,No Wind,Played
0,0.0,0.0,1.0,85,1.0,0.0
1,0.0,0.0,1.0,80,0.0,0.0
2,1.0,0.0,0.0,83,1.0,1.0
3,0.0,1.0,0.0,70,1.0,1.0
4,0.0,1.0,0.0,68,1.0,1.0
5,0.0,1.0,0.0,65,0.0,0.0
6,1.0,0.0,0.0,64,0.0,1.0
7,0.0,0.0,1.0,72,1.0,0.0
8,0.0,0.0,1.0,69,1.0,1.0
9,0.0,1.0,0.0,75,1.0,1.0


In [95]:
#şimdi öncelikle test train kümelerini oluşturalım
#bağımsız değişkenler tüm tablo
#bağımlı değişken = humidity
x_train, x_test, y_train, y_test = train_test_split(gereklidf, hum, test_size=0.33, random_state=0)

print(x_train)
print("\n\n", y_train)
print("\n\n",x_test)
print("\n\n",y_test)

    overcast  rainy  sunny  temperature  No Wind  Played
13       0.0    1.0    0.0           71      0.0     0.0
9        0.0    1.0    0.0           75      1.0     1.0
1        0.0    0.0    1.0           80      0.0     0.0
7        0.0    0.0    1.0           72      1.0     0.0
10       0.0    0.0    1.0           75      0.0     1.0
3        0.0    1.0    0.0           70      1.0     1.0
0        0.0    0.0    1.0           85      1.0     0.0
5        0.0    1.0    0.0           65      0.0     0.0
12       1.0    0.0    0.0           81      1.0     1.0


     humidity
13        91
9         80
1         90
7         95
10        70
3         96
0         85
5         70
12        75


     overcast  rainy  sunny  temperature  No Wind  Played
8        0.0    0.0    1.0           69      1.0     1.0
6        1.0    0.0    0.0           64      0.0     1.0
4        0.0    1.0    0.0           68      1.0     1.0
11       1.0    0.0    0.0           72      0.0     1.0
2        

In [96]:
#şimdi ise regresyon uygulamalarını yapıp tahminle gerçeği karşılaştıralım
lr = LinearRegression()
lr.fit(x_train, y_train)
y_predict = lr.predict(x_test)
print(y_test)
print(y_predict)

    humidity
8         70
6         65
4         80
11        90
2         86
[[84.45365573]
 [63.93839954]
 [85.76050662]
 [64.21013241]
 [75.06793322]]


In [97]:
#Şimdi ise Backward Elimination yöntemi ile sistemi iyileştirmeye çalışalım
X = np.append(arr=np.ones((14,1)).astype(int), values=gereklidf, axis=1)
xl = gereklidf.iloc[:, [0, 1, 2, 3, 4, 5]].values
xl = np.array(xl, dtype=float)

model = sm.OLS(endog = hum, exog = xl).fit()
print(model.summary())

#en yüksek x5

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.294
Model:                            OLS   Adj. R-squared:                 -0.148
Method:                 Least Squares   F-statistic:                    0.6653
Date:                Sun, 15 Nov 2020   Prob (F-statistic):              0.661
Time:                        01:24:41   Log-Likelihood:                -49.542
No. Observations:                  14   AIC:                             111.1
Df Residuals:                       8   BIC:                             114.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            48.3605     46.536      1.039      0.3

C:\Users\beray\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=14
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [98]:
xl = gereklidf.iloc[:, [0, 1, 2, 3, 5]].values
xl = np.array(xl, dtype=float)

model = sm.OLS(endog = hum, exog = xl).fit()
print(model.summary())

#en yüksek x3

                            OLS Regression Results                            
Dep. Variable:               humidity   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                 -0.060
Method:                 Least Squares   F-statistic:                    0.8165
Date:                Sun, 15 Nov 2020   Prob (F-statistic):              0.546
Time:                        01:24:49   Log-Likelihood:                -49.809
No. Observations:                  14   AIC:                             109.6
Df Residuals:                       9   BIC:                             112.8
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1            36.6564     39.905      0.919      0.3

In [99]:
#Şu an p-value'lar çok yüksek olduğu için x_trainden ve x_testten, üst kısımda attığımız kolonu çıkarıyoruz
#Daha sonra tekrardan lineer işlemlerini yapıyoruz
atilacak = x_train.iloc[:, 4:5]
sol = x_train.iloc[:, :4]
sag = x_train.iloc[:, 5:]

x_train = pd.concat([sol, sag], axis=1)

atilacak = x_test.iloc[:, 4:5]
sol = x_test.iloc[:, :4]
sag = x_test.iloc[:, 5:]

x_test = pd.concat([sol, sag], axis=1)

lr.fit(x_train, y_train)
y_predict = lr.predict(x_test)
print(y_test)
print(y_predict)

#buradan da görülüyor ki sistemde iyileşme var
#Önceden ilk değeri 84 olarak tahmin ederken şu an 81 olarak tahmin ediyoruz mesela

    humidity
8         70
6         65
4         80
11        90
2         86
[[77.98135142]
 [68.29304916]
 [81.05037539]
 [71.44926132]
 [75.78905304]]
